<a href="https://colab.research.google.com/github/benjipeng/Colab-Projects/blob/main/pyTorch_Learn_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `PyTorch` All-In-One

## Basic APIs

### Start
> A most basic example, highligting the overall syntax

Note: Every `TorchVision` Dataset includes two arguments: `transform` and `target_transform` to modify the ***samples*** and ***labels*** respectively.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="FashionMNIST", # Saved folder name
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="FashionMNIST",
    train=False,
    download=True,
    transform=ToTensor(),
)

**What's next after downloading the `Dataset`**

`dataset` is an argument to the `DataLoader`, therefore, define a `batch_size`

In [12]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)
for X, y in test_dataloader: # Inspection
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [25]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: FashionMNIST
    Split: Train
    StandardTransform
Transform: ToTensor()

**Create `Models`**

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"{device} is available, using {device} device")

cpu is avaliable, using cpu device


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device) # remember this

In [19]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


1. **`loss function` and `optimizer`**
2. Create a `training loop`

In [36]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    model.train #Set the module in training mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_function(pred, y)
        # Back Prop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad() #Resets the gradients of all optimized torch.Tensor s

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #torch.no_grad() ensures that no gradients are computed during test mode
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


The training process

In [32]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    test(test_dataloader, model, loss_function)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309656  [   64/60000]
loss: 2.300575  [ 6464/60000]
loss: 2.281383  [12864/60000]
loss: 2.283329  [19264/60000]
loss: 2.248446  [25664/60000]
loss: 2.235044  [32064/60000]
loss: 2.242405  [38464/60000]
loss: 2.212619  [44864/60000]
loss: 2.216372  [51264/60000]
loss: 2.185622  [57664/60000]
Test Error: 
 Accuracy: 36.5%, Avg loss: 2.180316 

Epoch 2
-------------------------------
loss: 2.193666  [   64/60000]
loss: 2.187094  [ 6464/60000]
loss: 2.136517  [12864/60000]
loss: 2.151797  [19264/60000]
loss: 2.092989  [25664/60000]
loss: 2.050664  [32064/60000]
loss: 2.074626  [38464/60000]
loss: 2.009214  [44864/60000]
loss: 2.014082  [51264/60000]
loss: 1.940784  [57664/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 1.942819 

Epoch 3
-------------------------------
loss: 1.980675  [   64/60000]
loss: 1.951112  [ 6464/60000]
loss: 1.845504  [12864/60000]
loss: 1.877147  [19264/60000]
loss: 1.755396  [25664/60000]
loss: 1.719320  [32064/600

`Save` and `Load` Models

In [33]:
torch.save(model.state_dict(), "model.pth")

In [34]:
model = NeuralNetwork().to(device) #Remember that
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [35]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


### `torch.autograd`

In [37]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [38]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7d085d284730>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7d085d286fb0>


In [41]:
print(w.grad)
print(b.grad)
loss.backward()
print(w.grad)
print(b.grad)

None
None
tensor([[0.2959, 0.0406, 0.0350],
        [0.2959, 0.0406, 0.0350],
        [0.2959, 0.0406, 0.0350],
        [0.2959, 0.0406, 0.0350],
        [0.2959, 0.0406, 0.0350]])
tensor([0.2959, 0.0406, 0.0350])


In [44]:
t = torch.tensor([[[1, 2],
                  [3, 4]],
                  [[5, 6],
                  [7, 8]]])

In [50]:
t.shape

torch.Size([2, 2, 2])

In [51]:
torch.flatten(t, start_dim=0, end_dim=1).shape

torch.Size([4, 2])